---
title: Electron Wavefunctions
authors: [gvarnavides]
date: 2025-01-22
---

In [118]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import display, clear_output
import ipywidgets

import abtem
import py4DSTEM # for visualization

## Apertures

In [124]:
def return_probe_arrays(semiangle_cutoff):
    """ """
    probe = abtem.Probe(
        semiangle_cutoff=semiangle_cutoff,
        sampling=(0.125,0.125),
        gpts=(128,128),
        energy=300e3,
    ).build(
        (0,0), # corner-centered
        lazy=False
    )
    
    complex_probe = np.fft.fftshift(probe.array)
    fourier_probe = probe.diffraction_patterns(
        return_complex=True,
        max_angle=None,
    ).array
    return probe, fourier_probe, complex_probe

probe, fourier_probe, complex_probe  = return_probe_arrays(
    semiangle_cutoff=20,
)

dpi = 72
with plt.ioff():
    fig,axs = plt.subplots(1,2,figsize=(675/dpi,325/dpi),dpi=dpi)

bar_common = {"Nx":128,"Ny":128,"labelsize":8}

rgb_fourier_probe = py4DSTEM.visualize.Complex2RGB(
    fourier_probe,
    vmin=0,vmax=1,
)
im_fourier = axs[0].imshow(rgb_fourier_probe)
bar_fourier= bar_common | {"pixelsize":probe.angular_sampling[0],"pixelunits":"mrad"}

rgb_complex_probe = py4DSTEM.visualize.Complex2RGB(
    complex_probe,
    vmin=0,vmax=1,
)
im_complex = axs[1].imshow(rgb_complex_probe)
bar_complex= bar_common | {"pixelsize":probe.sampling[0],"pixelunits":r"$\AA$"}

bars = [bar_fourier, bar_complex]
titles= ["Fourier-space probe", "real-space probe"]

for ax, bar, title in zip(axs, bars, titles):
    ax.set(xticks=[],yticks=[],title=title)
    divider = make_axes_locatable(ax)
    ax_cb = divider.append_axes("right", size="5%", pad="2.5%")
    py4DSTEM.visualize.add_colorbar_arg(ax_cb)
    py4DSTEM.visualize.add_scalebar(ax,bar)

fig.tight_layout()
fig.canvas.resizable = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = True
fig.canvas.layout.width = '675px'
fig.canvas.layout.height = '350px'
fig.canvas.toolbar_position = 'bottom'

In [137]:
def update_aperture(change):
    """ """
    semiangle_cutoff = change['new']
    probe, fourier_probe, complex_probe  = return_probe_arrays(
        semiangle_cutoff=semiangle_cutoff,
    )

    rgb_fourier_probe = py4DSTEM.visualize.Complex2RGB(
        fourier_probe,
        vmin=0,vmax=1,
    )
    im_fourier.set_data(rgb_fourier_probe)
    
    rgb_complex_probe = py4DSTEM.visualize.Complex2RGB(
        complex_probe,
        vmin=0,vmax=1,
    )
    im_complex.set_data(rgb_complex_probe)
    fig.canvas.draw_idle()
    return None

style = {
    'description_width': 'initial',
}

layout = ipywidgets.Layout(width='450px',height='30px')

semiangle_slider = ipywidgets.FloatSlider(
    min=5,
    max=50,
    step=0.5,
    value=20,
    layout=layout,
    style=style,
    description="convergence semi-angle [mrad]",
) 
semiangle_slider.observe(update_aperture, 'value')

In [138]:
#| label: app:convergence_angle_widget
display(
    ipywidgets.VBox(
        [
            semiangle_slider,
            fig.canvas
        ],
        layout=ipywidgets.Layout(
            align_items="center"
        )
    )
)